In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv("cleansed_data.csv")
forecast_data = pd.read_csv("forecast_data.csv")

In [3]:
raw_data.head()


,date,dc_name,size_code,retail_price,total_tires,zip_code,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,Key,category,day_name
0,2022-02-05,OAKLAND,1856015,70.090,1,94604,17.5,4.8,0.0,7.4,OAKLAND|1856015,Lumpy,Saturday
1,2021-03-02,OAKLAND,1856015,59.090,1,94604,20.3,6.3,0.0,12.9,OAKLAND|1856015,Lumpy,Tuesday
2,2020-12-08,OAKLAND,1856015,53.545,1,94604,23.6,5.6,0.0,10.0,OAKLAND|1856015,Lumpy,Tuesday
3,2022-01-17,OAKLAND,1856015,58.410,1,94604,15.8,6.8,0.0,7.4,OAKLAND|1856015,Lumpy,Monday
4,2020-11-13,OAKLAND,1856015,43.085,1,94604,16.9,7.5,0.0,21.5,OAKLAND|1856015,Lumpy,Friday


In [4]:
forecast_data['Key'] = forecast_data['dc_name'] + '|'+forecast_data['size_code'].astype(str)
forecast_data.head()

,date,dc_name,size_code,Key
0,2022-09-20,BAKERSFIELD,1856015,BAKERSFIELD|1856015
1,2022-09-21,BAKERSFIELD,1856015,BAKERSFIELD|1856015
2,2022-09-22,BAKERSFIELD,1856015,BAKERSFIELD|1856015
3,2022-09-23,BAKERSFIELD,1856015,BAKERSFIELD|1856015
4,2022-09-24,BAKERSFIELD,1856015,BAKERSFIELD|1856015


In [5]:
raw_data = raw_data[raw_data['dc_name']=='BAKERSFIELD']
train_data = raw_data[['date','Key','total_tires']]
train_data['date'] = pd.to_datetime(train_data['date'])
forecast_data['date'] = pd.to_datetime(forecast_data['date'])

/var/folders/94/qbcd9c954nnf3wh_kk6kyj380000gn/T/ipykernel_40064/3763851938.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['date'] = pd.to_datetime(train_data['date'])


In [6]:
from sklearn.linear_model import LinearRegression


In [7]:
train_data

,date,Key,total_tires
76018,2021-04-07,BAKERSFIELD|1856015,1
76019,2021-04-10,BAKERSFIELD|1856015,1
76020,2022-01-28,BAKERSFIELD|1856015,1
76021,2022-01-18,BAKERSFIELD|1856015,1
76022,2021-02-19,BAKERSFIELD|1856015,1
...,...,...,...
171545,2022-07-24,BAKERSFIELD|3512520,0
171546,2020-11-22,BAKERSFIELD|3512520,0
171547,2021-08-08,BAKERSFIELD|3512520,0
171548,2022-02-14,BAKERSFIELD|3512520,0


In [8]:
def linear_reg(sliced_data,future_data):


    key = sliced_data['Key'].unique()[0]
    #future_data = forecast_data.copy()

    test_df = future_data[future_data['Key'] == key]
    
    x_train = sliced_data['date'].values.reshape(-1, 1)
    x_train = x_train.astype(float)
    y_train = sliced_data['total_tires'].values

    x_test = test_df['date'].values.reshape(-1, 1)
    x_test = x_test.astype(float)

    
    regressor = LinearRegression() 

    regressor.fit(x_train,y_train)
    

    y_test = regressor.predict(x_test)
    
    test_df['Forecast'] = y_test.round()
    test_df['Model'] = 'Linear_Regression'
    test_df['Type'] = 'Univariate'
    test_df = test_df.drop(columns='Key')
    return test_df


In [9]:
py_af_output = train_data.groupby('Key').apply(linear_reg,forecast_data).reset_index()

In [10]:
py_af_output

,Key,level_1,date,dc_name,size_code,Forecast,Model,Type
0,BAKERSFIELD|1856015,0,2022-09-20,BAKERSFIELD,1856015,8.0,Linear_Regression,Univariate
1,BAKERSFIELD|1856015,1,2022-09-21,BAKERSFIELD,1856015,8.0,Linear_Regression,Univariate
2,BAKERSFIELD|1856015,2,2022-09-22,BAKERSFIELD,1856015,8.0,Linear_Regression,Univariate
3,BAKERSFIELD|1856015,3,2022-09-23,BAKERSFIELD,1856015,8.0,Linear_Regression,Univariate
4,BAKERSFIELD|1856015,4,2022-09-24,BAKERSFIELD,1856015,8.0,Linear_Regression,Univariate
...,...,...,...,...,...,...,...,...
730,BAKERSFIELD|3512520,730,2022-09-22,BAKERSFIELD,3512520,15.0,Linear_Regression,Univariate
731,BAKERSFIELD|3512520,731,2022-09-23,BAKERSFIELD,3512520,15.0,Linear_Regression,Univariate
732,BAKERSFIELD|3512520,732,2022-09-24,BAKERSFIELD,3512520,15.0,Linear_Regression,Univariate
733,BAKERSFIELD|3512520,733,2022-09-25,BAKERSFIELD,3512520,15.0,Linear_Regression,Univariate


In [11]:
linear_result = py_af_output[['date','dc_name','size_code','Forecast']].rename(columns={'Forecast':'total_tires'})
linear_result

,date,dc_name,size_code,total_tires
0,2022-09-20,BAKERSFIELD,1856015,8.0
1,2022-09-21,BAKERSFIELD,1856015,8.0
2,2022-09-22,BAKERSFIELD,1856015,8.0
3,2022-09-23,BAKERSFIELD,1856015,8.0
4,2022-09-24,BAKERSFIELD,1856015,8.0
...,...,...,...,...
730,2022-09-22,BAKERSFIELD,3512520,15.0
731,2022-09-23,BAKERSFIELD,3512520,15.0
732,2022-09-24,BAKERSFIELD,3512520,15.0
733,2022-09-25,BAKERSFIELD,3512520,15.0


In [12]:
pip install AutoTS

Note: you may need to restart the kernel to use updated packages.


In [13]:
from autots import AutoTS

In [14]:
model = AutoTS(
    forecast_length=15,
    frequency='D',
    prediction_interval=0.95,
    ensemble=None,
    models_mode='deep',
    model_list = 'univariate',# or ['ARIMA','ETS']
    max_generations=10,
    num_validations=3,
    no_negatives=True,
    n_jobs='auto')

Using 6 cpus for n_jobs.


In [15]:
from autots.models.model_list import model_lists
print(model_lists.keys())

dict_keys(['all', 'default', 'fast', 'superfast', 'parallel', 'fast_parallel', 'probabilistic', 'multivariate', 'univariate', 'no_params', 'recombination_approved', 'no_shared', 'no_shared_fast', 'experimental', 'slow', 'gpu', 'regressor', 'best', 'motifs'])


In [16]:
print(model_lists['univariate'])

['Greykite', 'Theta', 'UnivariateMotif', 'ConstantNaive', 'UnobservedComponents', 'NeuralProphet', 'DatepartRegression', 'ARDL', 'ETS', 'AverageValueNaive', 'ARIMA', 'GLS', 'UnivariateRegression', 'FBProphet', 'ARCH', 'SeasonalNaive', 'GLM', 'LastValueNaive']


In [17]:
def auto_ml(sliced_data,future_data):
    model = AutoTS(
    forecast_length=7,
    frequency='D',
    prediction_interval=0.95,
    ensemble=None,
    models_mode='deep',
    model_list = 'univariate',# or ['ARIMA','ETS']
    max_generations=5,
    num_validations=3,
    no_negatives=True,
    n_jobs='auto')

    key = sliced_data['Key'].unique()[0]

    test_df = future_data[future_data['Key'] == key]
    
    sliced_data.set_index('date', inplace=True)
    model.fit(sliced_data['total_tires'])
    prediction = model.predict(forecast_length=7)
    
    test_df['Forecast'] = prediction.forecast['total_tires'].values.tolist()
    test_df['Forecast'] = test_df['Forecast'].round()
    test_df['Model'] = 'Auto_ML_Best_Model'
    test_df['Type'] = 'Univariate'
    test_df = test_df.drop(columns='Key')
    return test_df

In [ ]:
auto_output = train_data.groupby('Key').apply(auto_ml,forecast_data).reset_index()

Using 6 cpus for n_jobs.
Model Number: 1 with model ARIMA in generation 0 of 5
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           12     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.97323D+00    |proj g|=  1.20620D+00

At iterate    1    f=  3.53429D+00    |proj g|=  4.39921D-01

At iterate    2    f=  3.43637D+00    |proj g|=  2.17070D-01

At iterate    3    f=  3.39211D+00    |proj g|=  1.44728D-01

At iterate    4    f=  3.36964D+00    |proj g|=  6.46714D-02

At iterate    5    f=  3.35758D+00    |proj g|=  3.20759D-02

At iterate    6    f=  3.35602D+00    |proj g|=  7.66416D-02

At iterate    7    f=  3.35367D+00    |proj g|=  3.26259D-02

At iterate    8    f=  3.35124D+00    |proj g|=  3.12122D-02



 This problem is unconstrained.


At iterate    9    f=  3.34901D+00    |proj g|=  3.17992D-02

At iterate   10    f=  3.34619D+00    |proj g|=  2.53916D-02

At iterate   11    f=  3.34549D+00    |proj g|=  1.70390D-02

At iterate   12    f=  3.34475D+00    |proj g|=  1.96952D-02

At iterate   13    f=  3.34433D+00    |proj g|=  1.56180D-02

At iterate   14    f=  3.34376D+00    |proj g|=  1.82951D-02

At iterate   15    f=  3.34234D+00    |proj g|=  2.56485D-02

At iterate   16    f=  3.34156D+00    |proj g|=  2.07408D-02

At iterate   17    f=  3.34117D+00    |proj g|=  5.11773D-03

At iterate   18    f=  3.34109D+00    |proj g|=  5.90675D-03

At iterate   19    f=  3.34099D+00    |proj g|=  5.12907D-03

At iterate   20    f=  3.34072D+00    |proj g|=  8.44677D-03

At iterate   21    f=  3.34051D+00    |proj g|=  4.20333D-02

At iterate   22    f=  3.34015D+00    |proj g|=  3.25235D-02

At iterate   23    f=  3.33993D+00    |proj g|=  1.26851D-02

At iterate   24    f=  3.33966D+00    |proj g|=  1.02423D-02

At itera


At iterate  144    f=  3.32303D+00    |proj g|=  1.51792D-02

At iterate  145    f=  3.32303D+00    |proj g|=  8.61796D-03

At iterate  146    f=  3.32302D+00    |proj g|=  3.53235D-03

At iterate  147    f=  3.32302D+00    |proj g|=  2.25512D-02

At iterate  148    f=  3.32302D+00    |proj g|=  3.28627D-03

At iterate  149    f=  3.32302D+00    |proj g|=  1.23004D-02

At iterate  150    f=  3.32302D+00    |proj g|=  3.12462D-02

At iterate  151    f=  3.32301D+00    |proj g|=  1.57306D-02

At iterate  152    f=  3.32301D+00    |proj g|=  1.51296D-02

At iterate  153    f=  3.32301D+00    |proj g|=  1.38037D-02

At iterate  154    f=  3.32301D+00    |proj g|=  1.08674D-02

At iterate  155    f=  3.32301D+00    |proj g|=  1.93117D-02

At iterate  156    f=  3.32301D+00    |proj g|=  2.21288D-02

At iterate  157    f=  3.32301D+00    |proj g|=  2.38507D-03

At iterate  158    f=  3.32300D+00    |proj g|=  1.94319D-02

At iterate  159    f=  3.32300D+00    |proj g|=  1.78720D-02

At iter


At iterate  277    f=  3.32206D+00    |proj g|=  1.03746D-01

At iterate  278    f=  3.32204D+00    |proj g|=  5.38404D-02

At iterate  279    f=  3.32201D+00    |proj g|=  4.15168D-02

At iterate  280    f=  3.32200D+00    |proj g|=  7.81953D-02

At iterate  281    f=  3.32198D+00    |proj g|=  2.10554D-02

At iterate  282    f=  3.32195D+00    |proj g|=  2.18220D-02

At iterate  283    f=  3.32194D+00    |proj g|=  1.28902D-02

At iterate  284    f=  3.32192D+00    |proj g|=  1.08803D-01

At iterate  285    f=  3.32192D+00    |proj g|=  1.44289D-02

At iterate  286    f=  3.32192D+00    |proj g|=  1.46483D-02

At iterate  287    f=  3.32192D+00    |proj g|=  3.72793D-02

At iterate  288    f=  3.32191D+00    |proj g|=  7.26295D-02

At iterate  289    f=  3.32190D+00    |proj g|=  1.09860D-01

At iterate  290    f=  3.32188D+00    |proj g|=  1.40236D-01

At iterate  291    f=  3.32184D+00    |proj g|=  1.31694D-01

At iterate  292    f=  3.32184D+00    |proj g|=  1.04350D-01

At iter

Model Number: 2 with model ARIMA in generation 0 of 5
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            9     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.40483D+00    |proj g|=  1.10902D-02

At iterate    1    f=  3.40477D+00    |proj g|=  7.25224D-03

At iterate    2    f=  3.40472D+00    |proj g|=  6.83739D-03

At iterate    3    f=  3.40458D+00    |proj g|=  7.18597D-03

At iterate    4    f=  3.40446D+00    |proj g|=  8.05033D-03

At iterate    5    f=  3.40430D+00    |proj g|=  7.10671D-03

At iterate    6    f=  3.40427D+00    |proj g|=  1.44187D-02

At iterate    7    f=  3.40415D+00    |proj g|=  5.67737D-03

At iterate    8    f=  3.40405D+00    |proj g|=  4.90562D-03

At iterate    9    f=  3.40387D+00    |proj g|=  7.94812D-03

At iterate   10    f=  3.40372D+00    |proj g|=  7.53931D-03

At iterate   11    f=  3.40363D+00    |proj g|=  4.00163D-03

At iterate   12    f=  3.40359D+00

 This problem is unconstrained.



At iterate   17    f=  3.40358D+00    |proj g|=  1.89744D-04

At iterate   18    f=  3.40358D+00    |proj g|=  1.58973D-04

At iterate   19    f=  3.40358D+00    |proj g|=  1.94493D-04

At iterate   20    f=  3.40358D+00    |proj g|=  2.09486D-04

At iterate   21    f=  3.40358D+00    |proj g|=  7.98265D-05

At iterate   22    f=  3.40358D+00    |proj g|=  1.70100D-05

At iterate   23    f=  3.40358D+00    |proj g|=  1.14121D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    9     23     26      1     0     0   1.141D-05   3.404D+00
  F =   3.4035817422264030     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH   

 This problem is unconstrained.



At iterate    1    f=  6.10665D+00    |proj g|=  1.06407D+01

At iterate    2    f=  5.49230D+00    |proj g|=  1.22218D+01

At iterate    3    f=  5.32045D+00    |proj g|=  5.00974D+00

At iterate    4    f=  4.77443D+00    |proj g|=  3.71651D+00

At iterate    5    f=  4.49769D+00    |proj g|=  4.41825D+00

At iterate    6    f=  4.26313D+00    |proj g|=  1.74396D+00

At iterate    7    f=  4.05296D+00    |proj g|=  9.99996D-01

At iterate    8    f=  3.79205D+00    |proj g|=  5.83685D-01

At iterate    9    f=  3.65521D+00    |proj g|=  3.90125D-01

At iterate   10    f=  3.60914D+00    |proj g|=  1.49151D+00

At iterate   11    f=  3.54304D+00    |proj g|=  3.77728D-01

At iterate   12    f=  3.52599D+00    |proj g|=  3.72771D-01

At iterate   13    f=  3.48927D+00    |proj g|=  3.13945D-01

At iterate   14    f=  3.44883D+00    |proj g|=  2.83633D-01

At iterate   15    f=  3.42183D+00    |proj g|=  3.50242D-01

At iterate   16    f=  3.40739D+00    |proj g|=  2.71662D-01

At iter

In [ ]:
auto_output = auto_output[['date','dc_name','size_code','Forecast']].rename(columns={'Forecast':'total_tires'})
auto_output

In [ ]:
#import requests
#url = "https://scoring-app-uuzeqpiufa-ue.a.run.app/forecast/validate"
#payload = {
  
#"team_key": "9MaRyrU5",
#"data": df_submission.to_dict(orient="records")}
#response = requests.post(url, json=payload) 
#print(response.status_code, response.content)